In [127]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from joblib import dump, load
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.set_default_dtype(torch.float32)

%matplotlib qt

In [128]:
filetype = '_3_withpos'
trajectories = sio.loadmat("./Dataset/original_data" + filetype + ".mat")['training_matrix']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [129]:
class ToyMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(ToyMLP, self).__init__()
        self.fc = nn.Sequential(nn.Linear(input_size, 128),
                                nn.ReLU(),
                                nn.Linear(128, 128),
                                nn.ReLU(),
                                nn.Linear(128, 128),
                                nn.ReLU(),
                                nn.Linear(128, 64),
                                nn.ReLU(),
                                nn.Linear(64, 32),
                                nn.ReLU(),
                                nn.Linear(32, output_size)
                                )
    def forward(self, x):
        out = self.fc(x)
        return out

# LSTM Baseline
class ToyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ToyLSTM, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()

    def forward(self, x):
        batch_size = x.shape[0]
        out = self.activation(self.linear1(x))
        _, (hn, cn) = self.lstm(out.view(batch_size, 1, -1))
        out = self.activation(self.linear2(hn[0]))
        out = self.linear3(out)
        return out

class ToyGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ToyGRU, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(input_size=hidden_size, hidden_size=hidden_size, num_layers=2,batch_first=True)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()

    def forward(self, x):
        batch_size = x.shape[0]
        out = self.activation(self.linear1(x))
        _, hn = self.gru(out.view(batch_size, 1, -1))
        out = self.activation(self.linear2(hn[0]))
        out = self.linear3(out)
        return out

class ProLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ProLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2, bidirectional=True, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.PReLU()

    def forward(self, x):
        batch_size = x.shape[0]
        out, (hn, cn) = self.lstm(x.view(batch_size, 1, -1))
        out = self.activation(self.linear1(hn[0])) # self.linear1(out)
        out = self.activation(self.linear2(out))
        out = self.linear3(out)
        return out

# Expressive GRU
class ProGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ProGRU, self).__init__()
        self.lstm = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=2, bidirectional=True, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.PReLU()

    def forward(self, x):
        batch_size = x.shape[0]
        out, hn = self.lstm(x.view(batch_size, 1, -1))
        out = self.activation(self.linear1(hn[0])) # self.linear1(out)
        out = self.activation(self.linear2(out))
        out = self.linear3(out)
        return out

In [130]:
input_size = 18
hidden_size = 512
output_size = 1
interval = 1000
start = 1
end = start + interval

model_mlp = ToyMLP(input_size, output_size)
model_mlp.load_state_dict(torch.load('./Weights/weights_mlp' + filetype + '.pth'))
model_mlp.eval()

model_gru = ProGRU(input_size, hidden_size, output_size)
model_gru.load_state_dict(torch.load('./Weights/weights_gru' + filetype + '.pth'))
model_gru.eval()

model = ProLSTM(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('./Weights/weights_lstm' + filetype + '.pth'))
model.eval()

trajectories = torch.tensor(trajectories.astype(np.float32))

orientation_state = list([0])
X_state = list([0])
Y_state = list([0])
orientation_state_lstm = list([0])
X_state_lstm = list([0])
Y_state_lstm = list([0])
orientation_state_gru = list([0])
X_state_gru = list([0])
Y_state_gru = list([0])
orientation_state_mlp = list([0])
X_state_mlp = list([0])
Y_state_mlp = list([0])
deltaT = 0.05
vels = trajectories[:,3]

for i in range(start,end):

    omega = trajectories[i-1,-1]
    orientation = orientation_state[i-start] + deltaT * omega
    X = (X_state[i-start] + deltaT* np.cos(orientation_state[i-start])*vels[i-start].numpy()).item()
    Y = (Y_state[i-start] + deltaT* np.sin(orientation_state[i-start])*vels[i-start].numpy()).item()
    X_state.append(X), Y_state.append(Y), orientation_state.append(orientation)


trajectories_norm = (trajectories[:,:-1] - trajectories[:,:-1].mean(axis=0)) / trajectories[:,:-1].std(axis=0)

X = trajectories_norm.numpy()
with torch.no_grad():
    omega_lstm = model(trajectories_norm.reshape(trajectories_norm.shape[0], input_size))
    omega_mlp = model_mlp(trajectories_norm)
    omega_gru = model_gru(trajectories_norm.reshape(trajectories_norm.shape[0], input_size))



for i in range(start,end):

    omega = omega_lstm[i-1]

    orientation = orientation_state_lstm[i-start] + deltaT * omega

    X = (X_state_lstm[i-start] + deltaT* np.cos(orientation_state_lstm[i-start])*vels[i-start].numpy()).item()
    Y = (Y_state_lstm[i-start] + deltaT* np.sin(orientation_state_lstm[i-start])*vels[i-start].numpy()).item()
    X_state_lstm.append(X), Y_state_lstm.append(Y), orientation_state_lstm.append(orientation)

for i in range(start,end):

    omega = omega_mlp[i-1]

    orientation = orientation_state_mlp[i-start] + deltaT * omega

    X = (X_state_mlp[i-start] + deltaT* np.cos(orientation_state_mlp[i-start])*vels[i-start].numpy()).item()
    Y = (Y_state_mlp[i-start] + deltaT* np.sin(orientation_state_mlp[i-start])*vels[i-start].numpy()).item()
    X_state_mlp.append(X), Y_state_mlp.append(Y), orientation_state_mlp.append(orientation)


for i in range(start,end):

    omega = omega_gru[i-1]

    orientation = orientation_state_gru[i-start] + deltaT * omega

    X = (X_state_gru[i-start] + deltaT* np.cos(orientation_state_gru[i-start])*vels[i-start].numpy()).item()
    Y = (Y_state_gru[i-start] + deltaT* np.sin(orientation_state_gru[i-start])*vels[i-start].numpy()).item()
    X_state_gru.append(X), Y_state_gru.append(Y), orientation_state_gru.append(orientation)


plt.figure(1)
plt.plot(X_state, Y_state, label='truth')
plt.plot(X_state_lstm, Y_state_lstm, label='lstm')
plt.plot(X_state_gru, Y_state_gru, label='gru')
plt.plot(X_state_mlp, Y_state_mlp, label='mlp')
plt.legend(loc='best')
plt.show()